In [1]:
pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 990.6 kB/s eta 0:00:11
   - -------------------------------------- 0.5/11.0 MB 990.6 kB/s eta 0:00:11
   -- ------------------------------------- 0.8/11.0 MB 852.4 kB/s eta 0:00:12
   -- ------------------------------------- 0.8/11.0 MB 852.4 kB/s eta 0:00:12
   --- ------------------------------------ 1.0/11.0 MB 755.3 kB/s eta 0:00:14
   ----- ---------------------------------- 1.6/11.0 MB 915.4 kB/s eta 0:00:11
   --------- ------------------------------ 2.6/11.0 MB 1.4 MB/s eta 0:00:06
   --------------- -----------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv
from groq import Groq
import sqlalchemy

# Loading variables from the .env file
load_dotenv()

print("Libraries loaded and environment variables are ready.")

Libraries loaded and environment variables are ready.


In [6]:
# Getting the database URL from environment variables
db_url = os.getenv("DB_URL")

try:
    # Creating a SQLAlchemy engine to connect to the database
    engine = sqlalchemy.create_engine(db_url)
    
    # Testing the connection by fetching the first customer
    with engine.connect() as connection:
        result = connection.execute(sqlalchemy.text("SELECT name FROM customers LIMIT 1;"))
        for row in result:
            print(f"Successfully connected to the database. Found customer: {row.name}")

except Exception as e:
    print(f"Failed to connect to the database. Error: {e}")

Failed to connect to the database. Error: (psycopg2.OperationalError) could not translate host name "77049@localhost" to address: Non-recoverable failure in name resolution

(Background on this error at: https://sqlalche.me/e/20/e3q8)
